In [1]:
#import needed package
#---------------------------------------------------------
import pandas as pd
import numpy as np
import sklearn
import autosklearn
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import datetime
import autosklearn.classification
from sklearn.preprocessing import LabelEncoder #OrdinalEncoder

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#print versions
#---------------------------------------------------------
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The Auto scikit-learn version is {}.'.format(autosklearn.__version__))

The scikit-learn version is 0.19.2.
The Auto scikit-learn version is 0.5.2.


In [3]:
#Load Data from CSV
#--------------------------------------------------------
filepath = '/data/home/eissa/Downloads/datasets_84-BayesianNetworkGenerator_anneal.ORIG_small.csv'
rawdata = pd.read_csv( filepath ,  sep=','  ,  low_memory=False )
rawdata.shape

(1000000, 39)

In [4]:
rawdata.head(10)

,family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,...,s,p,shape,thick,width,len,oil,bore,packing,class
0,TN,C,A,'\'B1of3\'','\'B1of3\'',T,S,3,'\'B1of3\'',N,...,Y,Y,COIL,'\'B1of3\'','\'B1of3\'','\'B1of3\'',N,0,3,3
1,TN,C,K,'\'B2of3\'','\'B1of3\'',T,S,2,'\'B2of3\'',N,...,Y,Y,COIL,'\'B1of3\'','\'B2of3\'','\'B1of3\'',N,600,3,3
2,TN,C,A,'\'B1of3\'','\'B2of3\'',T,S,2,'\'B3of3\'',N,...,Y,Y,COIL,'\'B2of3\'','\'B3of3\'','\'B1of3\'',Y,0,3,3
3,TN,C,A,'\'B3of3\'','\'B1of3\'',T,S,3,'\'B1of3\'',N,...,Y,Y,SHEET,'\'B1of3\'','\'B3of3\'','\'B1of3\'',Y,0,3,5
4,TN,C,R,'\'B1of3\'','\'B1of3\'',T,S,2,'\'B1of3\'',N,...,Y,Y,SHEET,'\'B1of3\'','\'B3of3\'','\'B1of3\'',Y,0,3,3
5,TN,C,R,'\'B1of3\'','\'B1of3\'',T,S,2,'\'B1of3\'',N,...,Y,Y,SHEET,'\'B1of3\'','\'B2of3\'','\'B3of3\'',Y,0,3,3
6,TN,C,A,'\'B1of3\'','\'B1of3\'',T,S,2,'\'B1of3\'',N,...,Y,Y,SHEET,'\'B1of3\'','\'B3of3\'','\'B3of3\'',Y,0,3,3
7,TN,C,R,'\'B1of3\'','\'B1of3\'',T,S,2,'\'B1of3\'',N,...,Y,Y,SHEET,'\'B1of3\'','\'B2of3\'','\'B3of3\'',Y,0,3,3
8,TN,C,M,'\'B1of3\'','\'B1of3\'',T,S,2,'\'B3of3\'',N,...,Y,Y,COIL,'\'B1of3\'','\'B2of3\'','\'B1of3\'',Y,0,3,2
9,TN,C,R,'\'B1of3\'','\'B1of3\'',T,S,2,'\'B1of3\'',N,...,Y,Y,SHEET,'\'B2of3\'','\'B2of3\'','\'B3of3\'',Y,0,3,2


In [5]:
#Categorical feature encoding
#--------------------------------------------------------
label = 'class'
le = LabelEncoder()
for column in rawdata.columns: 
    rawdata[column] = le.fit_transform(rawdata[column])

In [6]:
rawdata.head(10)

,family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,...,s,p,shape,thick,width,len,oil,bore,packing,class
0,3,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,2,2
1,3,0,1,1,0,0,1,1,1,0,...,0,0,0,0,1,0,0,2,2,2
2,3,0,0,0,1,0,1,1,2,0,...,0,0,0,1,2,0,1,0,2,2
3,3,0,0,2,0,0,1,2,0,0,...,0,0,1,0,2,0,1,0,2,4
4,3,0,3,0,0,0,1,1,0,0,...,0,0,1,0,2,0,1,0,2,2
5,3,0,3,0,0,0,1,1,0,0,...,0,0,1,0,1,2,1,0,2,2
6,3,0,0,0,0,0,1,1,0,0,...,0,0,1,0,2,2,1,0,2,2
7,3,0,3,0,0,0,1,1,0,0,...,0,0,1,0,1,2,1,0,2,2
8,3,0,2,0,0,0,1,1,2,0,...,0,0,0,0,1,0,1,0,2,1
9,3,0,3,0,0,0,1,1,0,0,...,0,0,1,1,1,2,1,0,2,1


In [7]:
# split data into Testing and training
# ---------------------------------------------------------------------------------------------------------------------
np.random.seed(1234)
msk = np.random.rand(len(rawdata)) < 0.8
training_data = rawdata[msk]
Testing_data = rawdata[~msk]

X_train = training_data.loc[:, training_data.columns != label]
Y_train = training_data.loc[:, training_data.columns == label]

X_test = Testing_data.loc[:, Testing_data.columns != label]
Y_test = Testing_data.loc[:, Testing_data.columns == label]

In [9]:
# auto ML
# ---------------------------------------------------------------------------------------------------------------------
t1 = datetime.datetime.now()
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=300, 
                                                          ensemble_size= 1,
                                                          include_preprocessors=["no_preprocessing"],
                                                          ensemble_memory_limit=45000, 
                                                          seed=1, 
                                                          ml_memory_limit=45000)
automl.fit(X_train, Y_train)
y_hat = automl.predict(X_test)
t2 = datetime.datetime.now()
print("Time:" + str((t2 - t1).total_seconds()))
print("Accuracy score", accuracy_score(Y_test, y_hat))


/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/automl.py:887: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Will change shape via np.ravel().
  y = self._check_y(y)
/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2020-01-06 11:07:41,798:EnsembleBuilder(1):b8a65e9003175f67a14b241508fce21d] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 11:07:41,866:EnsembleBuilder(1):b8a65e9003175f67a14b241508fce21d] No models better than random - using Dummy Score!
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (297.023665)
[WARNING] [2020-01-06 11:07:43,877:EnsembleBuilder(1):b8a65e9003175f67a14b241508fce21d] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 11:07:45,889:EnsembleBuilder(1):b8a65e9003175f67a14b241508fce21d] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 11:07:47,900:EnsembleBuilder(1):b8a65e9003175f67a14b241508fce21d] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 11:07:49,912:EnsembleBuilder(1):b8a65e9003175f67a14b241508fce21d] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 11:07:51,

/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


1
['/tmp/autosklearn_tmp_29153_9752/.auto-sklearn/ensembles/1.0000000000.ensemble', '/tmp/autosklearn_tmp_29153_9752/.auto-sklearn/ensembles/1.0000000001.ensemble', '/tmp/autosklearn_tmp_29153_9752/.auto-sklearn/ensembles/1.0000000002.ensemble']
Time:302.941006
Accuracy score 0.884357449702692


In [ ]:
automl.cv_results_

In [ ]:
automl.show_models(

In [ ]:
automl.sprint_statistics()

In [ ]:
print("Accuracy score", accuracy_score(Y_test, y_hat))